In [70]:
import requests

resp = requests.get('https://indiapicororno.herokuapp.com/statesstatus')
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

In [71]:
response_dict = resp.json()
response_df = pd.DataFrame.from_records(response_dict)
response_df.head()

,0,1,2,3,4,5
0,1,Andhra Pradesh,3,0,0,0
1,2,Chhattisgarh,1,0,0,0
2,3,Delhi,16,1,5,1
3,4,Gujarat,5,0,0,0
4,5,Haryana,3,14,0,0


In [72]:
indian_states_df = pd.read_csv('../data/Indian_States.csv')
indian_states_df.head()

,State,lat,lon
0,Rajasthan,27.0238,74.2179
1,Kerala,10.8505,76.2711
2,Maharashtra,19.7515,75.7139
3,Goa,15.2993,74.1240
4,Union Territory of Jammu and Kashmir,33.7782,76.5762


In [73]:
def read_state_info(row):
    state = row['State']
    target_row = response_df[response_df[1] == state]
    if not target_row.empty:
        row['TotalConfirmedCasesInIndia'] = int(target_row[2].values[0])
        row['TotalConfirmedCasesForeignNational'] = int(target_row[3].values[0])
        row['Cured'] = int(target_row[4].values[0])
        row['Death'] = int(target_row[5].values[0])
    else:
        row['TotalConfirmedCasesInIndia'] = 0
        row['TotalConfirmedCasesForeignNational'] = 0
        row['Cured'] = 0
        row['Death'] = 0
    
    return row

indian_states_corona_df = indian_states_df.apply(lambda x: read_state_info(x), axis=1)
indian_states_corona_df = indian_states_corona_df.sort_values('TotalConfirmedCasesInIndia', ascending=False)
indian_states_corona_df.head()

,State,lat,lon,TotalConfirmedCasesInIndia,TotalConfirmedCasesForeignNational,Cured,Death
2,Maharashtra,19.7515,75.7139,49,3,0,1
1,Kerala,10.8505,76.2711,26,2,3,0
8,Uttar Pradesh,26.8467,80.9462,22,1,9,0
0,Rajasthan,27.0238,74.2179,15,2,3,0
5,Karnataka,15.3173,75.7139,15,0,1,1


In [74]:
import pandas as pd

import plotly.express as px

fig = px.scatter_mapbox(indian_states_corona_df, lat="lat", lon="lon", hover_name="State", hover_data=["TotalConfirmedCasesInIndia", "TotalConfirmedCasesForeignNational", "Cured", "Death"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [75]:
import plotly.express as px
fig = px.scatter_geo(indian_states_corona_df, lat='lat', lon='lon', color="State",
                     hover_name="State", size="TotalConfirmedCasesInIndia", locationmode='ISO-3',
                     projection="natural earth")
fig.show()